In [1]:
import numpy as np
np.random.seed(1712)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
# import statistics
import tensorflow as tf

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
# from tf.keras.models import Sequenti
# from tf.keras.layers.core import Dense, Dropout, Activation, Flatten
# from tf.keras.layers.convolutional import Convolution2D, MaxPooling2D
# from tf.keras.optimizers import SGD
# from tf.keras.utils import np_utils
# from tf.keras.models import model_from_json
from sklearn.metrics import log_loss
# from scipy.misc import imread, imresize
from numpy.random import permutation
# import tf.keras.layers

/home/roman/anaconda2/envs/gpu/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
use_cache = 1
# color type: 1 - grey, 3 - rgb
color_type_global = 3

In [3]:
def get_driver_data():
    driver_dict = dict()
    path = os.path.join('..', 'dataset', 'driver_imgs_list.csv')
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')
        driver_dict[arr[2]] = arr[0]
    f.close()
    return driver_dict

In [4]:
# 使用cv2將圖片resize
def get_im_cv2(path, img_rows, img_cols, color_type=1):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    mean_pixel = [103.939, 116.799, 123.68]
#     resized = resized.astype(np.float32, copy=False)
#     resized = resized[:, :, [2, 1, 0]] # swap chanel from RGB to BGR

#     for c in range(3):
#         resized[:, :, c] = resized[:, :, c] - mean_pixel[c]
#     resized = resized.transpose((2, 0, 1))
#     resized = np.expand_dims(img, axis=0)
    return resized

In [5]:
def load_train(img_rows, img_cols, color_type=1):
    X_train = []
    y_train = []
    driver_id = []

    driver_data = get_driver_data()

    print('Read train images')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join('..', 'dataset', 'train_1', 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl, img_rows, img_cols, color_type)
            X_train.append(img)
            y_train.append(j)
            driver_id.append(driver_data[flbase])

    unique_drivers = sorted(list(set(driver_id)))
    print('Unique drivers: {}'.format(len(unique_drivers)))
    print(unique_drivers)
    return X_train, y_train, driver_id, unique_drivers

In [6]:
# 加载测试集
def load_test(img_rows, img_cols, color_type=1):
    print('Read test images')
    # 读取测试数据
    path = os.path.join('..', 'input', 'imgs', 'test', '*.jpg')
    # 找到path下的所有jpg文件
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0

    # 所有jpg文件总数除以10向下取整
    thr = math.floor(len(files)/10)
    for fl in files:
        # 获取图片名，xxxx.jpg
        flbase = os.path.basename(fl)
        # 缩减图片大小
        img = get_im(fl, img_rows, img_cols, color_type)
        # 将图片加入测试集
        X_test.append(img)
        # 将图片对应的名称加入X_test_id
        X_test_id.append(flbase)
        # 累计数加1
        total += 1
        # 如果读到批数，输出从xxxx个文件中读取xxxx张图
        if total % thr == 0:
            print('Read {} images from {}'.format(total, len(files)))
    # 返回测试集与测试集图片对应的名称
    return X_test, X_test_id

In [6]:
# 缓存图片（数据，路径）
def cache_data(data, path):
    # 如果cache目录不存在
    if not os.path.isdir('cache'):
        # 创建cache目录
        os.mkdir('cache')
    # 如果路径存在
    if os.path.isdir(os.path.dirname(path)):
        # 打开文件
        file = open(path, 'wb')
        # 将data持久保存到file中
        pickle.dump(data, file)
        # 关闭文件流
        file.close()
        print('cache finish')
    else:
        print('Directory doesnt exists')


In [7]:
# 从缓存读取数据
def restore_data(path):
    # 创建data字典
    data = dict()
    # 如果文件存在
    if os.path.isfile(path):
        print('Restore data from pickle........')
        file = open(path, 'rb')
        # 从文件中读取数据
        data = pickle.load(file)
    print('restore finish')
    return data

In [8]:
def read_and_normalize_train_data(img_rows, img_cols, color_type=1,):
    cache_path = os.path.join('cache', 'train_r_' + str(img_rows) + '_c_' + str(img_cols) + '_t_' + str(color_type) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        train_data, train_target, driver_id, unique_drivers = load_train(img_rows, img_cols, color_type)
        train_data = np.array(train_data, dtype=np.uint8)
        train_target = np.array(train_target, dtype=np.uint8)
        if color_type == 1:
            train_data = train_data.reshape(train_data.shape[0], color_type,
                                            img_rows, img_cols)
        else:
            train_data = train_data.transpose((2, 0, 1, 3))
        train_target = tf.keras.utils.to_categorical(train_target, 10)
        train_data = train_data.astype('float32')
        mean_pixel = [103.939, 116.779, 123.68]
        for c in range(3):
            train_data[:, :, :, c] = train_data[:, :, :, c] - mean_pixel[c]
        # train_data /= 255
        perm = permutation(len(train_target))
        train_data = train_data[perm]
        train_target = train_target[perm]
        cache_data((train_data, train_target, driver_id, unique_drivers), cache_path)
    else:
        print('Restore train from cache!')
        (train_data, train_target, driver_id, unique_drivers) = restore_data(cache_path)
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, driver_id, unique_drivers

In [10]:
def read_and_normalize_test_data(img_rows=224, img_cols=224, color_type=1):
    cache_path = os.path.join('cache', 'test_r_' + str(img_rows) +
                              '_c_' + str(img_cols) + '_t_' +
                              str(color_type) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        test_data, test_id = load_test(img_rows, img_cols, color_type)
        cache_data((test_data, test_id), cache_path)
    else:
        print('Restore test from cache!')
        (test_data, test_id) = restore_data(cache_path)

    test_data = np.array(test_data, dtype=np.uint8)

    if color_type == 1:
        test_data = test_data.reshape(test_data.shape[0], color_type,
                                      img_rows, img_cols)
    else:
        test_data = test_data.transpose((0, 3, 1, 2))

    test_data = test_data.astype('float32')
    mean_pixel = [103.939, 116.779, 123.68]
    for c in range(3):
        test_data[:, c, :, :] = test_data[:, c, :, :] - mean_pixel[c]
    # test_data /= 255
    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    return test_data, test_id

In [9]:
def copy_selected_drivers(train_data, train_target, driver_id, driver_list):
    data = []
    target = []
    index = []
    for i in range(len(driver_id)):
        if driver_id[i] in driver_list:
            data.append(train_data[i])
            target.append(train_target[i])
            index.append(i)
    data = np.array(data, dtype=np.float32)
    target = np.array(target, dtype=np.float32)
    index = np.array(index, dtype=np.uint32)
    return data, target, index


In [10]:
def merge_several_folds_mean(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a += np.array(data[i])
    a /= nfolds
    return a.tolist()

In [11]:
def vgg_std16_model(img_rows, img_cols, color_type=1):
    print("color_type=%d,img_rows=%d,img_cols=%d"%(color_type, img_rows, img_cols))
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.ZeroPadding2D((1, 1), input_shape=(img_cols, img_rows, color_type)))
    model.add(tf.keras.layers.Convolution2D(64, 3, 3, activation='relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(64, 3, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_last'))

    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(128, 3, 3, activation='relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(128, 3, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_last'))

    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(256, 3, 3, activation='relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(256, 3, 3, activation='relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(256, 3, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_last'))

    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(512, 3, 3, activation='relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(512, 3, 3, activation='relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(512, 3, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_last'))

    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(512, 3, 3, activation='relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(512, 3, 3, activation='relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1, 1)))
    model.add(tf.keras.layers.Convolution2D(512, 3, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_last'))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(4096, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(4096, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1000, activation='softmax'))

    model.load_weights('../dataset/vgg16_weights.h5')

    # Code above loads pre-trained data and
    model.layers.pop()
    model.add(Dense(10, activation='softmax'))
    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')
    return model

In [12]:
def create_submission(predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = info + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)

In [ ]:
img_rows, img_cols = 224, 224
batch_size = 64
nb_epoch = 1
random_state = 51

train_data, train_target, driver_id, unique_drivers = read_and_normalize_train_data(img_rows, img_cols, color_type_global)


Read drivers data
Read train images
Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9
Unique drivers: 26
['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']


In [ ]:
# test_data, test_id = read_and_normalize_test_data(img_rows, img_cols, color_type_global)

In [16]:
yfull_train = dict()
yfull_test = []
unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                 'p050', 'p051', 'p052', 'p056']
unique_list_valid = ['p061', 'p064', 'p066', 'p072',
                     'p075', 'p081']

In [17]:
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)

In [18]:
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

In [26]:
print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)

Start Single Run
('Split train: ', 6130, 6130)
('Split valid: ', 1614, 1614)
('Train drivers: ', ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056'])
('Test drivers: ', ['p061', 'p064', 'p066', 'p072', 'p075', 'p081'])


In [36]:
model = vgg_std16_model(img_rows, img_cols, color_type_global)

color_type=3,img_rows=224,img_cols=224


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_15/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,256].

In [21]:
def run_single():
    # input image dimensions
    img_rows, img_cols = 224, 224
    batch_size = 64
    nb_epoch = 1
    random_state = 51

    train_data, train_target, driver_id, unique_drivers = read_and_normalize_train_data(img_rows, img_cols, color_type_global)
    test_data, test_id = read_and_normalize_test_data(img_rows, img_cols, color_type_global)

    yfull_train = dict()
    yfull_test = []
    unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p050', 'p051', 'p052', 'p056']
    X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
    unique_list_valid = ['p061', 'p064', 'p066', 'p072',
                     'p075', 'p081']
    X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

    print('Start Single Run')
    print('Split train: ', len(X_train), len(Y_train))
    print('Split valid: ', len(X_valid), len(Y_valid))
    print('Train drivers: ', unique_list_train)
    print('Test drivers: ', unique_list_valid)

    model = vgg_std16_model(img_rows, img_cols, color_type_global)
    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
              show_accuracy=True, verbose=1, validation_data=(X_valid, Y_valid))

    # score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
    # print('Score log_loss: ', score[0])

    predictions_valid = model.predict(X_valid, batch_size=64, verbose=1)
    score = log_loss(Y_valid, predictions_valid)
    print('Score log_loss: ', score)

    # Store valid predictions
    for i in range(len(test_index)):
        yfull_train[test_index[i]] = predictions_valid[i]

    # Store test predictions
    test_prediction = model.predict(test_data, batch_size=64, verbose=1)
    yfull_test.append(test_prediction)

    print('Final log_loss: {}, rows: {} cols: {} epoch: {}'.format(score, img_rows, img_cols, nb_epoch))
    info_string = 'loss_' + str(score) \
                    + '_r_' + str(img_rows) \
                    + '_c_' + str(img_cols) \
                    + '_ep_' + str(nb_epoch)

    test_res = merge_several_folds_mean(yfull_test, 1)
    print("Creating submission file")
    create_submission(test_res, test_id, info_string)

In [ ]:
run_single()

Read drivers data
Read train images
Load folder c0
Load folder c1
Load folder c2
Load folder c3
